In [1]:
import pandas as pd
import numpy as np

from sklearn import linear_model
from sklearn import tree
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures

Uploading competition data

In [2]:
trainX = pd.read_csv("competition_data/trainX.csv", index_col = 'Id')
trainY = pd.read_csv("competition_data/trainY.csv", index_col = 'Id')
testX = pd.read_csv("competition_data/testX.csv", index_col = 'Id')

trainY = trainY.Value.to_numpy()
trainX = trainX.to_numpy()
testX = testX.to_numpy()

Kaggle public test score: 0.64698

In [4]:
poly = PolynomialFeatures(2)
poly_X = poly.fit_transform(trainX)
poly_testX = poly.fit_transform(testX)

pa_regr = linear_model.PassiveAggressiveRegressor(max_iter=100, random_state=0, tol=1e-3)
pa_regr.fit(poly_X, trainY)
y_pred = pa_regr.predict(poly_testX).reshape(2000,)

Ypd = pd.DataFrame({'Value': y_pred, 'Id': range(len(y_pred))})
Ypd.to_csv('mullagaliev_amir.csv', index=False)
Ypd

,Value,Id
0,19.722460,0
1,34.838256,1
2,29.166753,2
3,27.638779,3
4,31.772254,4
...,...,...
1995,40.082500,1995
1996,39.433653,1996
1997,26.779717,1997
1998,32.181348,1998


Kaggle public test score: 0.43746

In [4]:
poly = PolynomialFeatures(2)
poly_X = poly.fit_transform(trainX)
poly_testX = poly.fit_transform(testX)

lin_regr = linear_model.LinearRegression()
lin_regr.fit(poly_X, trainY)
y_pred = lin_regr.predict(poly_testX).reshape(2000,)

Ypd = pd.DataFrame({'Value': y_pred, 'Id': range(len(y_pred))})
Ypd.to_csv('mullagaliev_amir.csv', index=False)
Ypd

,Value,Id
0,23.955078,0
1,32.609375,1
2,27.628906,2
3,26.041016,3
4,28.181641,4
...,...,...
1995,34.949219,1995
1996,37.283203,1996
1997,23.570312,1997
1998,28.683594,1998
